In [1]:
from pyspark.sql import SparkSession

# Initialize Spark session with Iceberg configurations
spark = SparkSession.builder \
  .appName("IcebergLocalDevelopment") \
  .config('spark.jars.packages', 'org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.6.1') \
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
  .config("spark.sql.catalog.local", "org.apache.iceberg.spark.SparkCatalog") \
  .config("spark.sql.catalog.local.type", "hadoop") \
  .config("spark.sql.catalog.local.warehouse", "spark-warehouse/iceberg") \
  .getOrCreate()

24/10/13 02:22:08 WARN Utils: Your hostname, ubuntulab resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
24/10/13 02:22:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/ubuntu/codigos/spark-iceberg/.venv/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.iceberg#iceberg-spark-runtime-3.5_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f58adf16-dc53-4b80-8470-75e5258ffb65;1.0
	confs: [default]
	found org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 in central
:: resolution report :: resolve 381ms :: artifacts dl 12ms
	:: modules in use:
	org.apache.iceberg#iceberg-spark-runtime-3.5_2.12;1.6.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   1   |   0   |   0   |   0   ||   1   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.s

In [2]:
spark

In [3]:
spark.sql(
  """
  CREATE TABLE local.carro_iceberg (id INT, placa STRING) USING iceberg
  """
)

DataFrame[]

In [4]:
spark.sql("select * from local.carro_iceberg").show()

+---+-----+
| id|placa|
+---+-----+
+---+-----+



In [5]:
spark.sql("INSERT INTO local.carro_iceberg VALUES (1, 'XYZ1J34'), (2, 'RLC5B93'), (3, 'ABV1V23')")

DataFrame[]

In [6]:
spark.sql("select * from local.carro_iceberg").show()

+---+-------+
| id|  placa|
+---+-------+
|  1|XYZ1J34|
|  2|RLC5B93|
|  3|ABV1V23|
+---+-------+



In [7]:
spark.sql(
    """
    alter table local.carro_iceberg add column marca STRING, modelo STRING, ano INT
    """
)

DataFrame[]

In [8]:
spark.sql(
    """
    select * from local.carro_iceberg
    """
).show()

+---+-------+-----+------+----+
| id|  placa|marca|modelo| ano|
+---+-------+-----+------+----+
|  1|XYZ1J34| NULL|  NULL|NULL|
|  2|RLC5B93| NULL|  NULL|NULL|
|  3|ABV1V23| NULL|  NULL|NULL|
+---+-------+-----+------+----+



In [9]:
spark.sql(
    """
    update local.carro_iceberg set marca = 'Renault', modelo = 'Sandero', ano = 2021 where id = 1
    """
)

DataFrame[]

In [10]:
spark.sql(
    """
    select * from local.carro_iceberg
    """
).show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  2|RLC5B93|   NULL|   NULL|NULL|
|  3|ABV1V23|   NULL|   NULL|NULL|
|  1|XYZ1J34|Renault|Sandero|2021|
+---+-------+-------+-------+----+



In [11]:
spark.sql('update local.carro_iceberg set marca="GM", modelo="tracker", ano=2020 where id = 2    ')

DataFrame[]

In [12]:
spark.sql('update local.carro_iceberg set marca="Ford", modelo="EcoSport", ano=2022 where id = 3    ')

DataFrame[]

In [13]:
spark.sql('select * from local.carro_iceberg').show()

+---+-------+-------+--------+----+
| id|  placa|  marca|  modelo| ano|
+---+-------+-------+--------+----+
|  2|RLC5B93|     GM| tracker|2020|
|  3|ABV1V23|   Ford|EcoSport|2022|
|  1|XYZ1J34|Renault| Sandero|2021|
+---+-------+-------+--------+----+



In [14]:
spark.sql('delete from local.carro_iceberg where id = 3')

DataFrame[]

In [15]:
spark.sql('select * from local.carro_iceberg').show()

+---+-------+-------+-------+----+
| id|  placa|  marca| modelo| ano|
+---+-------+-------+-------+----+
|  2|RLC5B93|     GM|tracker|2020|
|  1|XYZ1J34|Renault|Sandero|2021|
+---+-------+-------+-------+----+

